# Solution To Cipher 2

In [81]:
from collections import Counter
import numpy as np

In [48]:
with open('cipher_2.txt') as f:
    cipher_2 = f.read()

cipher_2 = cipher_2.replace("\n", "")

In [49]:
cipher_2

'M74PF4K4G26IYDUNGXMS_FM41HIK0SK#EKKOD2OKUZK#U__4YD7YKT84Z8ZZMFIQUK04WC6GMB4PIK8498KQME9YKTP076IZ4EHYYNO#G1M4344PF41VSD6OM0GYFXK8_7O4YQAMQJ8RWWT4YD7YVNK5G26Q9EQYIXOS0K#0AQQMPJO3XET4F7CLIY8T4B6Z38MYRG#5G8N4F78YEU2#9B4434UACSOC986Y04UKG4K#VKLOEFF8_KNCZ2_4I7IJG4054BM4A5UJGG##_7O41HIK0HK59_I_JQNM00Y0VKNXACUBG4ZW1__4FEUUKTUS16IT9QBC0G04ABMJMC8YJK8VSZ6X007YVN#0A0P4F78YHO#49KXGDJUMH4061D5KDIURJKY3EKIT#Q4JUU814C_KEI8B0G8_S70Y_HCNV4YTG1M294MYYU#YG8V4F78YUZ2REKWLMJBC0GX5_A0KM8U8NRY8WZ6OFQ4JN410G9IYEQUG0NK7WKJK_EGC0GM2AWQTF47YCR00GGQZ3Q4YXK#AGGWXF7SYRK#4476Z34UBKY13_Y#45K7EG4KCXBIT6Q4LF4Y1W76N00LRGJ8_S76OM0GYVUVRG2#44IU80SY49KLK78AFVL2ZGDPO96URQ40SWKPO8QCL0ZRSG6QJEJUMH4RW8KKN4B7PGT80XK2NACUFG4RO8KVO94UFKY8S1ZMYFQ78WMR5WB6KEF8AKGVZEKQYM7CEJR6C89WQ0DUMH4RSG1IYM8HTKZORG6M4FEUEQ4K#VK_K0QBGO4K#VKQ44DNCPJ854KLUMIIYQT85Z_6L4HMR0UZ14B#_98NW0NOC121KEQ4R0UXSG8N4F78YTU6O1KP_9JCLI4V0V0MYMMBGEN8QS76H0QLCCIRSVKNXACUFGXOC_76G9QBMWX8O3Z6GM74JH4LAGGIR68HE0GXRGGPO_7UFG4YP9WQT03UJGG3SGDW44DB8DO1CS##KDQNFG4V08C6U1QB

In [50]:
characters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ_#"

In [51]:
def index_of_coincidence(sequence):
    N = len(sequence)
    frequencies = Counter(sequence)
    ic = sum(f * (f - 1) for _, f in frequencies.items()) / (N * (N - 1))
    return ic

In [52]:
def calculate_frequencies_for_key_length(cipher, key_length):
    chunks = [cipher[i::key_length] for i in range(key_length)]
    ics = [index_of_coincidence(chunk) for chunk in chunks]
    avg_ic = np.mean(ics)
    return avg_ic

In [53]:
max_key_length = 20

In [54]:
ics_by_length = {l: calculate_frequencies_for_key_length(cipher_2, l) for l in range(1, max_key_length + 1)}
ics_by_length

{1: 0.031933635402918194,
 2: 0.03613415707300257,
 3: 0.04326738251745397,
 4: 0.04360910278745645,
 5: 0.03190299235463599,
 6: 0.05561066187481281,
 7: 0.031956161250357384,
 8: 0.04358462877851062,
 9: 0.043130950193187474,
 10: 0.0360628262744564,
 11: 0.031884342037122856,
 12: 0.07794377247466493,
 13: 0.03184261030930627,
 14: 0.03616835145273062,
 15: 0.0432694325226514,
 16: 0.04355895219322893,
 17: 0.031905674333201274,
 18: 0.05544101429849366,
 19: 0.03189194500744449,
 20: 0.04358633224282079}

In [55]:
def frequency_distribution(text):
    freq_dist = Counter(text)
    freq_dist = {char: freq_dist.get(char, 0) / len(text) for char in characters}
    return freq_dist

In [56]:
def caesar_shift(sequence, shift):
    shifted_sequence = ''
    for char in sequence:
        if char in characters:
            shifted_sequence += characters[(characters.index(char) - shift) % len(characters)]
        else:
            shifted_sequence += char
    return shifted_sequence

In [57]:
def chi_squared_statistic(text, expected_freq):
    observed_freq = frequency_distribution(text)
    chi_squared = 0
    for char in characters:
        observed = observed_freq[char]
        expected = expected_freq[char] if char in expected_freq else 0
        chi_squared += ((observed - expected) ** 2) / expected if expected else 0
    return chi_squared

In [65]:
english_letter_freq = {
    'E': 12.7, 'T': 9.1, 'A': 8.2, 'O': 7.5, 'I': 7.0, 'N': 6.7, 'S': 6.3, 'H': 6.1, 'R': 6.0,
    'D': 4.3, 'L': 4.0, 'C': 2.8, 'U': 2.8, 'M': 2.4, 'W': 2.4, 'F': 2.2, 'G': 2, 'Y': 2.0,
    'P': 1.9, 'B': 1.5, 'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.074
}
# Ref https://en.wikipedia.org/wiki/Letter_frequency

In [66]:
total_freq = sum(english_letter_freq.values())
english_letter_freq = {char: freq / total_freq for char, freq in english_letter_freq.items()}

In [76]:
def find_best_shift_for_column(column):
    chi_squared_by_shift = {}
    for shift in range(len(characters)):
        shifted_column = caesar_shift(column, shift)
        chi_squared = chi_squared_statistic(shifted_column, english_letter_freq)
        chi_squared_by_shift[shift] = chi_squared
    best_shift = min(chi_squared_by_shift, key=chi_squared_by_shift.get)
    return best_shift

In [77]:
key_length = 12
columns = [cipher_2[i::key_length] for i in range(key_length)]
best_shifts = [find_best_shift_for_column(column) for column in columns]
best_shifts

[24, 28, 32, 36, 2, 6, 10, 14, 18, 22, 8, 6]

In [78]:
def apply_shifts_to_columns(columns, shifts):
    decoded_columns = []
    for i, column in enumerate(columns):
        shift = shifts[i]
        decoded_column = caesar_shift(column, shift)
        decoded_columns.append(decoded_column)
    return decoded_columns

In [79]:
def interleave_columns(columns):
    interleaved_text = ''
    for i in range(len(columns[0])):
        for column in columns:
            if i < len(column):
                interleaved_text += column[i]
    return interleaved_text

In [80]:
decoded_columns = apply_shifts_to_columns(columns, best_shifts)
decoded_text = interleave_columns(decoded_columns)
decoded_text[:500]

'_HARD_AS_I_CAN_PERCEIVE_FROM_MANY_CIRCUMSTANCES_AND_IN_SHORT_POSSESSES_A_LARGE_STOCK_OF_INFORMATION_WHEN_HE_HEARD_THAT_I_AM_DRAWING_A_GOOD_DEAL_AND_THAT_I_KNOW_GREEK_TWO_WONDERFUL_THINGS_FOR_THIS_PART_OF_THE_COUNTRY_HE_CAME_TO_SEE_ME_AND_DISPLAYED_HIS_WHOLE_STORE_OF_LEARNING_FROM_BATTEAUX_TO_WOOD_FROM_DE_PILES_TO_WINKELMANN_HE_ASSURED_ME_HE_HAD_READ_THROUGH_THE_FIRST_PART_OF_SULTZERS_THEORY_AND_ALSO_POSSESSED_A_MANUSCRIPT_OF_HEYNES_WORK_ON_THE_STUDY_OF_THE_ANTIQUE_I_ALLOWED_IT_ALL_TO_PASS__I_HAV'